In [ ]:
import dwave.system
import numpy as np
import matplotlib.pyplot as plt
import src.particle_funcs.distance_matrix as distance_matrix
import src.leap_funcs.qubo.q_matrix as q_matrix

In [ ]:
num_particles = 7

use_dimod_exact_solver = False

part_coords_n = np.zeros((num_particles,2))
part_coords_nm1 = np.zeros((num_particles,2))

for i in range(np.shape(part_coords_n)[0]):
    part_coords_n[i,:] = [0, i]
    part_coords_nm1[i,:] = [0.5*1, i]

fig_initial, axs_initial = plt.subplots(1,1)
axs_initial.scatter(part_coords_n[:,0], part_coords_n[:,1], label="n")
axs_initial.scatter(part_coords_nm1[:,0], part_coords_nm1[:,1], label="n-1")
fig_initial.legend()
fig_initial.show()

correct_sol = np.zeros(num_particles*num_particles)
for i in range(1, num_particles+1):
    correct_sol[(i-1)*num_particles + i -1] = 1.

print(correct_sol)

In [ ]:
distances = distance_matrix.calc_phi_ij(part_coords_n, part_coords_nm1, type='tanh_minimum', tanh_min=0.1)
print(distances)
ids_remove = np.where(distances > 4)
print(ids_remove)
print(np.shape(ids_remove))
print(np.max(distances))

In [ ]:
Q_dist = q_matrix.q_dist(distances)
assert (Q_dist.transpose() == Q_dist).all(), 'Q_dist is not symmetric'
with np.printoptions(precision=3):
    print(Q_dist)
max_abs = np.max(Q_dist)
#Q_dist /= max_abs
#with np.printoptions(precision=3):
#    print(Q_dist)

In [ ]:
Q_part = q_matrix.q_part(np.shape(distances)[0])
assert (Q_part.transpose() == Q_part).all(), 'Q_part is not symmetric'
print(Q_part)

In [ ]:
Q_pos = q_matrix.q_pos(np.shape(distances)[0])
assert (Q_pos.transpose() == Q_pos).all(), 'Q_pos is not symmetric'
print(Q_pos)

In [ ]:
lm = np.max(Q_dist)
Q_array = Q_dist + 2*lm*(Q_part + Q_pos)
print('lambda Q_2 and Q_3: ', lm)

Q_array = Q_dist/np.min([x for x in Q_dist.flatten() if x > 0.0]) + Q_part + Q_pos


with np.printoptions(precision=3, suppress=True):
    print(Q_array)


In [ ]:
d = 3.5
Q_dist_rem, i_mats, j_mats = q_matrix.remove_large_distances(Q_dist, distances, d)
Q_part_rem, _, _ = q_matrix.remove_large_distances(Q_part, distances, d)
Q_pos_rem, _, _= q_matrix.remove_large_distances(Q_pos, distances, d)

lm_rem = np.max(Q_dist_rem)
Q_array_rem = Q_dist_rem + 100*lm_rem*(Q_part_rem + Q_pos_rem)
print('lambda Q_2 and Q_3: ', lm_rem)
with np.printoptions(precision=3, suppress=True):
    print(Q_array_rem)
print(np.max(Q_array_rem))

In [ ]:
Q_dict = q_matrix.Q_convert_to_dict(Q_array)
#Q_dict = q_matrix.Q_convert_to_dict(Q_array_rem)
print(Q_dict)

In [ ]:
if use_dimod_exact_solver:
    from dimod import ExactSolver

    sampler = ExactSolver()

    solution = sampler.sample_qubo(Q_dict)

In [ ]:
if use_dimod_exact_solver:
    print(solution)

In [ ]:
if use_dimod_exact_solver:
    print(solution.data_vectors.keys())
    #print(solution.data_vectors['energy'])
    sorted_abs_energies = np.sort(np.abs(solution.data_vectors['energy']))
    ids_sorted_abs_energies = np.argsort(np.abs(solution.data_vectors['energy']))
    ids_sorted_energies = np.argsort(solution.data_vectors['energy'])

    print(solution.record[ids_sorted_abs_energies[0:7]])
    print(solution.record[ids_sorted_energies[0:7]])

In [ ]:
if use_dimod_exact_solver:
    ids_correct_solution = []
    for i, sol in enumerate(solution.record):
        #print(sol[0])
        if np.array_equal(sol[0], correct_sol):
            ids_correct_solution.append(i)
    print(ids_correct_solution)
    print(solution.record[ids_correct_solution])


In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite

with open('API_Token.txt') as file:
    token = file.readline().rstrip()
    architecture = file.readline().rstrip()

sampler = DWaveSampler(token = token, architecture='pegasus')
print(DWaveSampler._get_solver(sampler))

sampler = EmbeddingComposite(sampler)
sample = sampler.sample_qubo(Q_dict, num_reads = 1000, auto_scale=True)

In [ ]:
print(sample)

In [ ]:
#usually is sorted for energies automatically
ids_sorted_energies = np.argsort(sample.data_vectors['energy'])

print(sample.record[ids_sorted_energies[0:7]])

In [ ]:
from src.leap_funcs.qubo import filter_samples

ids_valid_solutions= filter_samples.indices_of_valid_solutions(sample)
print(ids_valid_solutions)
print(sample.record[ids_valid_solutions])

In [ ]:
for sol in sample.record:
    if np.array_equal(correct_sol, sol[0]):
        print(sol)
for sol in sample.record[ids_valid_solutions]:
    if np.array_equal(correct_sol, sol[0]):
        print(sol)

In [ ]:
#print(np.sort(i_mats))
#print(np.sort(j_mats))
#print(np.intersect1d(i_mats, j_mats))
#tmp = sample.record[ids_sorted_energies[0]][0]
#tmp = np.insert(tmp, 6, 0)
#tmp = np.insert(tmp, 42, 0)
#tmp = np.insert(tmp, 48, 0)
#print(tmp)

print(sample.record[ids_sorted_energies[0]][0])
print(correct_sol)
print(np.array_equal(correct_sol, sample.record[ids_sorted_energies[0]][0]))
correct_sol - sample.record[ids_sorted_energies[0]][0]